# Simulating DGP 0

In [1]:
import sys
import pandas as pd
from pathlib import Path

project_root = Path.cwd()
while project_root != project_root.parent and not (project_root / "src").exists():
    project_root = project_root.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from src.simulation.dgp0 import Tier0Config, simulate_panel
from src.simulation.validation import plot_market_plotly
from src.data.feature_eng import feature_eng_syn

In [2]:
cfg = Tier0Config(T=180, burn_in=24)
df = simulate_panel(cfg, n_markets=50, seed=42)

df.head()

,market_id,t,S,c,p
0,0,0,1,0.091785,0.060889
1,0,1,1,0.074484,0.060124
2,0,2,1,0.055169,0.055855
3,0,3,1,0.057500,0.047379
4,0,4,1,0.069277,0.051012


In [3]:
plot_market_plotly(df, market_id=5)

In [4]:
plot_market_plotly(df, 10)

In [5]:
plot_market_plotly(df, market_id= 20)

In [6]:
df_L18 = pd.read_parquet("../data/processed_syn/synth_dgp0_windows_L18.parquet")

df_L18.head()

,market_id,window_start,window_end,window_length,Price 1,Price 2,Price 3,Price 4,Price 5,Price 6,...,Price 14,Price 15,Price 16,Price 17,Price 18,share_C,share_T,share_K,state_mode,is_pure_80
0,0,0,17,18,0.060889,0.060124,0.055855,0.047379,0.051012,0.050917,...,0.007404,-0.000546,-0.008006,-0.021978,-0.019690,0.111111,0.888889,0.0,1,1.0
1,0,1,18,18,0.060124,0.055855,0.047379,0.051012,0.050917,0.055131,...,-0.000546,-0.008006,-0.021978,-0.019690,-0.077646,0.166667,0.833333,0.0,1,1.0
2,0,2,19,18,0.055855,0.047379,0.051012,0.050917,0.055131,0.036140,...,-0.008006,-0.021978,-0.019690,-0.077646,-0.094838,0.222222,0.777778,0.0,1,0.0
3,0,3,20,18,0.047379,0.051012,0.050917,0.055131,0.036140,0.055256,...,-0.021978,-0.019690,-0.077646,-0.094838,-0.104940,0.277778,0.722222,0.0,1,0.0
4,0,4,21,18,0.051012,0.050917,0.055131,0.036140,0.055256,0.060538,...,-0.019690,-0.077646,-0.094838,-0.104940,-0.100550,0.333333,0.666667,0.0,1,0.0


In [7]:
feature_df = feature_eng_syn(df_L18)
feature_df.head()

,market_id,window_start,window_end,window_length,Price 1,Price 2,Price 3,Price 4,Price 5,Price 6,...,share_K,state_mode,is_pure_80,mean_change,volatility,CoV_change,zero_change_fraction,autocorr_change,kurtosis_change,max_abs_ret
0,0,0,17,18,0.060889,0.060124,0.055855,0.047379,0.051012,0.050917,...,0.0,1,1.0,-0.004740,0.010565,2.228472,0.117647,-0.077116,1.501094,0.029864
1,0,1,18,18,0.060124,0.055855,0.047379,0.051012,0.050917,0.055131,...,0.0,1,1.0,-0.008104,0.016308,2.012092,0.058824,-0.182795,4.389317,0.057957
2,0,2,19,18,0.055855,0.047379,0.051012,0.050917,0.055131,0.036140,...,0.0,1,0.0,-0.008864,0.016413,1.851337,0.058824,-0.020393,3.844113,0.057957
3,0,3,20,18,0.047379,0.051012,0.050917,0.055131,0.036140,0.055256,...,0.0,1,0.0,-0.008960,0.016415,1.831822,0.058824,-0.019512,3.798594,0.057957
4,0,4,21,18,0.051012,0.050917,0.055131,0.036140,0.055256,0.060538,...,0.0,1,0.0,-0.008915,0.016450,1.844898,0.058824,-0.049699,3.761500,0.057957


In [ ]:
FEATURES = ["mean_change","volatility","CoV_change",
            "zero_change_fraction","AR_1","kurtosis_change" ,
              "max_abs_ret", "level_vol", "pos_vol", "neg_vol", "AR_2",
               "price_range" "ratio" ]

pure = feature_df[feature_df["is_pure_80"] == 1]
pure.groupby("state_mode")[FEATURES].mean()

,mean_change,volatility,CoV_change,zero_change_fraction,autocorr_change,kurtosis_change,max_abs_ret
state_mode,,,,,,,
0,0.000002,0.034480,47.656607,0.027976,0.069947,0.516971,0.079250
1,-0.000022,0.020404,36.881200,0.047527,0.090492,0.306735,0.046291
2,0.000047,0.013017,35.270613,0.077360,0.011404,0.203280,0.029022
